<a href="https://colab.research.google.com/github/jamesdvance/DobermanLabs/blob/main/Personalized_VLM_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -q einops

In [3]:
import cv2
import os
import torch
from PIL import Image as pillow
from typing import Union
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from einops import rearrange, repeat
from huggingface_hub import login


In [ ]:
model_id = "vikhyatk/moondream2"
output_folder = "./drive/MyDrive/"

In [4]:
def extract_frames(video_path, output_dir, frame_rate=1)->os.PathLike:
  os.makedirs(output_dir, exist_ok=True)
  cap = cv2.VideoCapture(video_path)
  fps = cap.get(cv2.CAP_PROP_FPS)
  num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frame_interval = int(fps / frame_rate)
  frame_count = 0
  while True:
    ret, frame = cap.read()
    if not ret:
      break

    if frame_count % frame_interval == 0:
      frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
      cv2.imwrite(frame_filename, frame)

    frame_count += 1

  cap.release()

  return output_dir


In [5]:
def load_model(model_id):
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      low_cpu_mem_usage=True
  )
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  return model, tokenizer